In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

from functools import partial
from bayes_opt import BayesianOptimization

import numpy as np
import random
import sys
import os
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Reading dataset

In [2]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read().lower().split('\n')
poem_list = []
raw_text = ''
for j in range(len(text) + 1):
    if j == len(text):
        poem_list.append(raw_text)
    elif text[j] == '':
        if raw_text != '':
            poem_list.append(raw_text)
        raw_text = ''
        continue
    elif text[j][-1].isdigit():
        continue
    else:
        subsentence = text[j] + '\n'
        raw_text += subsentence  
print('Number of poems:', len(poem_list))

Number of poems: 154


## Helper function for creating sequences

In [3]:
# organize into sequences of characters
def create_sequence(raw_text, length, step):
    sequences = []
    next_chars = []
    for i in range(0, len(raw_text) - length, step):
        # select sequence of tokens
        seq = raw_text[i:i + length]
        # store
        sequences.append(seq)
        next_chars.append(raw_text[i + length])
    return sequences, next_chars

In [4]:
length = 40
step = 1
sequences = []
next_chars = []
for poem in poem_list:
    sub_sequences, sub_next_chars = create_sequence(poem, length, step)
    sequences += sub_sequences
    next_chars += sub_next_chars
print('Total Sequences: %d' % len(sequences))

Total Sequences: 88130


## Character Mappings and Inverse Mappings

In [5]:
poem_string = "".join(poem_list)
chars = sorted(list(set(poem_string)))
char_index_map = dict((c, i) for i, c in enumerate(chars))
index_char_map = dict((i, c) for i, c in enumerate(chars))
vocab_size = len(char_index_map)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 38


## Vectorization

In [6]:
X = np.zeros((len(sequences), length, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, char in enumerate(sentence):
        X[i, t, char_index_map[char]] = 1
    y[i, char_index_map[next_chars[i]]] = 1    

## Helper functions for building and fitting RNN Training Model

In [7]:
def build_model(LSTM_size, dropout_rate):
    model = Sequential()
    model.add(LSTM(LSTM_size, input_shape=(length, len(chars))))
    model.add(Dense(len(chars)))
    model.add(Dropout(dropout_rate))
    model.add(Activation('softmax'))
    return model

In [8]:
def fit_model(LSTM_size, verbose, dropout_rate, lr):

    # Create the model using a specified hyperparameters.
    model = build_model(LSTM_size, dropout_rate)

    # Train the model for a specified number of epochs.
    optimizer = RMSprop(lr)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Train the model with the train dataset.
    model.fit(X, y, batch_size=128, epochs=10, verbose=verbose)

    # Evaluate the model with the eval dataset.
    score = model.evaluate(X, y, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Return the accuracy.

    return score[1]

verbose = 1
LSTM_size = 128
fit_with_partial = partial(fit_model, LSTM_size, verbose)

## Bayesian Optimization (tuning hyperparameters)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# Bounded region of parameter space
pbounds = {'dropout_rate': (0.1, 0.5), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=10,)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   | dropou... |    lr     |
-------------------------------------------------




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





88130/88130 [==============================] - 52s 596us/step - loss: 2.5321 - acc: 0.3069
Epoch 2/10
88130/88130 [==============================] - 52s 592us/step - loss: 2.2037 - acc: 0.3887
Epoch 3/10
88130/88130 [==============================] - 52s 596us/step - loss: 2.1136 - acc: 0.4129
Epoch 4/10
88130/88130 [==============================] - 55s 624us/step - loss: 2.0516 - acc: 0.4327
Epoch 5/10
88130/88130 [==============================] - 58s 662us/step - loss: 2.0092 - acc: 0.4432
Epoch 6/10
88130/88130 [==============================] - 61s 689us/step - loss: 1.9751 - acc: 0.4531
Epoch 7/10
88130/88130 [==============================] - 61s 698